In [1]:
!python -m pip install tensorflow-metal==0.8.0 tensorflow-macos==2.12.0 

In [3]:
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Dropout, Embedding
from tensorflow.keras.layers import Flatten, Input, AlphaDropout, GlobalMaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import os
import numpy as np
# os.environ['TF_USE_LEGACY_KERAS'] = '1'

class my_cnn(object):

    def __init__(self,input_size, optimizer='adam',loss='binary_crossentropy',num_classes=2, embedding_size=128):
        self.input_size = input_size
        self.max_index = 65535
        self.embedding_size = embedding_size
        self.optimizer = optimizer
        self.loss = loss
        self.metrics = ["accuracy"]
        self.num_classes = num_classes
        self.model =  self._build_model()
        print(self.model.summary())

    def _build_model(self):
        #        verbose, epochs, batch_size = 0, 1, 32
        model = Sequential()
        n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], self.num_classes
        model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps, n_features)))
        model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
        model.add(Dropout(0.2))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Flatten())
        model.add(Dense(100, activation='relu'))
        model.add(Dense(n_outputs, activation='sigmoid'))
        model.compile(loss='categorical_crossentropy', optimizer=self.optimizer, metrics=['accuracy'])

        #        model = Model(inputs = inputs, outputs = outputs)

        #        model.compile(optimizer = self.optimizer, loss = self.loss, metrics = self.metrics)
        return model


    def train(self, X_train, y_train, X_val, y_val, epochs, batch_size, checkpoint_every=None):
        self.model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size)

    def get_model(self):
        return self.model

    def test(self, X_test, y_test, batch_size):
        self.model.evaluate(X_test, y_test, batch_size=batch_size)

    def predict(self, X_test, batch_size):
        predict_x=self.model.predict(X_test,batch_size)
        classes_x=np.argmax(predict_x,axis=1)
        return classes_x

    def save_model(self, path):
        self.model.save(path)


In [4]:
from time import process_time

def testgpu():
    t0 = process_time()
    import tensorflow as tf
    tf.config.list_physical_devices()
    cifar = tf.keras.datasets.cifar100
    (x_train, y_train), (x_test, y_test) = cifar.load_data()
    model = tf.keras.applications.ResNet50(
        include_top=True,
        weights=None,
        input_shape=(32, 32, 3),
        classes=100,)
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
    model.fit(x_train, y_train, epochs=5, batch_size=64)
    t1 = process_time()
    print("Total time with gpu: ", t1-t0)

def testcpu():
    t0 = process_time()
    import tensorflow as tf
    with tf.device('/CPU:0'):
        tf.config.list_physical_devices()
        cifar = tf.keras.datasets.cifar100
        (x_train, y_train), (x_test, y_test) = cifar.load_data()
        model = tf.keras.applications.ResNet50(
            include_top=True,
            weights=None,
            input_shape=(32, 32, 3),
            classes=100,)
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
        model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
        model.fit(x_train, y_train, epochs=5, batch_size=64)
        t1 = process_time()
        print("Total time with cpu: ", t1-t0)

if __name__ == '__main__':
    testgpu()
    testcpu()

Metal device set to: Apple M1 Pro
Epoch 1/5


2024-09-03 08:46:22.865737: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


782/782 [==============================] - 65s 76ms/step - loss: 4.7200 - accuracy: 0.0775
Epoch 2/5
782/782 [==============================] - 56s 71ms/step - loss: 4.1737 - accuracy: 0.1221
Epoch 3/5
782/782 [==============================] - 56s 71ms/step - loss: 3.7016 - accuracy: 0.1806
Epoch 4/5
782/782 [==============================] - 58s 74ms/step - loss: 3.9176 - accuracy: 0.1498
Epoch 5/5
782/782 [==============================] - 57s 73ms/step - loss: 4.3038 - accuracy: 0.0947
Total time with gpu:  269.052398
Epoch 1/5
782/782 [==============================] - 388s 494ms/step - loss: 4.4707 - accuracy: 0.0937
Epoch 2/5
782/782 [==============================] - 384s 491ms/step - loss: 4.4305 - accuracy: 0.0930
Epoch 3/5
782/782 [==============================] - 386s 493ms/step - loss: 4.2079 - accuracy: 0.0938
Epoch 4/5
782/782 [==============================] - 390s 498ms/step - loss: 4.0388 - accuracy: 0.1022
Epoch 5/5
782/782 [==============================] - 395s 50

In [10]:
DATA_DIR = 'data/'
from glob import glob
all_csv_files = glob(DATA_DIR + "/*.csv")
all_csv_files

['data/heart_disease_health_indicators_BRFSS2015.csv']

In [11]:
import pandas as pd
for file in all_csv_files:
    data = pd.read_csv(file, on_bad_lines='warn')
data.head()

,HeartDiseaseorAttack,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


In [12]:
for col in data.columns:
    data[col] = data[col].apply(lambda x: float(x) if str(x).replace('.', '', 1).isdigit() else None)

In [14]:
data=data.dropna()
data.shape

(253680, 22)

In [15]:
data.columns

Index(['HeartDiseaseorAttack', 'HighBP', 'HighChol', 'CholCheck', 'BMI',
       'Smoker', 'Stroke', 'Diabetes', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education',
       'Income'],
      dtype='object')

In [25]:
def load_training_data(df):
    from sklearn.model_selection import train_test_split
    import pandas as pd
    c_label = 'HeartDiseaseorAttack'

    y = df[c_label].values
    X = df.loc[:, df.columns != c_label]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=13)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = load_training_data(data)


In [26]:
import numpy as np
loaded = list()
loaded.append(X_train)
# print(loaded)
X_train = np.dstack(loaded)
loaded = list()
loaded.append(X_test)
X_test = np.dstack(loaded)
X_train.shape

X_train = np.asarray(X_train).astype('float32')
X_test = np.asarray(X_test).astype('float32')

In [27]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
model = my_cnn(input_size=None)
# X_train = np.dstack(X_train)
# X_test = np.dstack(X_test)
model.train(X_train, y_train, X_test, y_test, epochs=5, batch_size=300)
m = model.get_model()



Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_6 (Conv1D)           (None, 19, 64)            256       
                                                                 
 conv1d_7 (Conv1D)           (None, 17, 64)            12352     
                                                                 
 dropout_3 (Dropout)         (None, 17, 64)            0         
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 8, 64)            0         
 1D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 512)               0         
                                                                 
 dense_6 (Dense)             (None, 100)               51300     
                                                      

In [ ]:
y_pred = model.predict(X_test=X_test,batch_size=300)

In [ ]:
y_pred

In [ ]:
## Evaluate Results
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
print('Accuracy: ', np.round(accuracy_score(y_test[:,1], y_pred),3))
print('F1 Score: ', np.round(f1_score(y_test[:,1], y_pred),3))
print('Precision: ', np.round(precision_score(y_test[:,1], y_pred),3))
print('Recall: ', np.round(recall_score(y_test[:,1], y_pred),3))
print('***************')